#### Feed Forward Neural Network Mathematics

The following is a programme that uses the neural network structure on the MNIST dataset in order to classify handwritten numbers. 
Although there are several python libraries that have in-built neural network functions, to aid in my understanding of the technique, I have coded all elements of a NN to run the MNIST classifier.

Below is some of the mathematics behind NN to aid in the explaination of what each function does. The reason for the success of NN in approximating non-linear functions lies in the proof by George Cybenko of the Universal Approximation Theorem.

##### Mathematics behind NN for Matrix Inputs

The following is the set up for a feed forward neural network that uses matrix inputs. 

- number of layers: $L$
- width of layers: $\eta_0,...,\eta_L$
- input layer: $X^{(0)} \in \mathbb{R}^{\eta_0 \times n}$; $n$ observations stacked in columns
- other layers: $X^{(l)} \in \mathbb{R}^{\eta_{l}\times n}$, where $X^{(l)} = \sigma^{(l)}\left(A^{(l)}X^{(l-1)} + B^{(l)}\right)$
- weight matrix: $A^{(l)} \in \mathbb{R}^{\eta_l \times \eta_{l-1}}$; the rows of $A^{(l)}$ are the weight vectors corresponding to the $\eta_l$ nodes in the $l$-th layer

We will aim to walkthrough the mathematics of the NN by an explaination of the forward and backpropogation processes as well as the gradient descent optimisation process through an example.

##### Example: 3 Layer Neural Network Matrix Input

Forward Propagation: 
1. $X^{(0)} \in \mathbb{R}^{\eta_0 \times n}$

2. $X^{(1)} = \sigma^{(1)}\left(A^{(1)}X^{(0)} + B^{(1)}\right)$

    $\sigma^{(1)} := $ Piecewise ReLU (Rectified Linear Unit),

    $\sigma^{(1)}(z_{ij}) = 
    \begin{cases}
    z_{ij} & \text{if } z_{ij} >0 \\
    0 & \text{if } z_{ij} \leq 0 
    \end{cases}$

3. $X^{(2)} = \sigma^{(2)}\left(A^{(2)}X^{(1)} + B^{(2)}\right)$

    $\sigma^{(2)} := $ Piecewise Softmax,

    $\sigma^{(2)}(z_{ij}) = \frac{e^{z_{ij}}}{\sum^{\eta_2}_{k = 1}e^{z_{kj}}}$

Backpropagation:

The process of backpropagation enables us to optimise the NN on the trainable parameters, the weight and biases for each layer of the NN. This begins by quantifying the misaccuracy of our classification NN. We use a loss function for which we then use chain-rule to find the gradient of the loss function with respect to our trainable parameters. We can then carry out gradient descent. 

Gradient Descent:

$A^{(l)} = A^{(l)} - \alpha \frac{\partial L}{\partial A^{(l)}}$

$B^{(l)} = B^{(l)} - \alpha \frac{\partial L}{\partial B^{(l)}}$

To calculate the derivative of the loss function with respect to the weights and biases, we use the chain rule. The steps are as follows:

1. Loss function: Cross entropy loss

    $L(X,Y) = -\frac{1}{n}\sum_{j=1}^{n}\sum_{i=1}^{\eta_2}y_{ij}\cdot log(x_{ij})$

2. Chain rule steps:

    $\frac{\partial L}{\partial A^{(2)}} = \frac{\partial{L}}{\partial X^{(2)}} \cdot \frac{\partial X^{(2)}}{\partial Z^{(2)}} \cdot \frac{\partial Z^{(2)}}{\partial A^{(2)}}$

    $\frac{\partial L}{\partial x^{(2)}}_{lk} = \frac{\partial L}{\partial x_{lk}} = -\frac{1}{n}\sum_{j=1}^{n}\sum_{i=1}^{\eta_2}y_{ij}\cdot \frac{\partial log(x_{ij})}{\partial x_{lk}} = -\frac{1}{n}\sum_{j=1}^{n}\sum_{i=1}^{\eta_2}\frac{y_{ij}}{x_{ij}}$

    $\frac{\partial X^{(2)}_{ij}}{\partial Z^{(2)}_{lk}} = \frac{\partial}{\partial z^{(2)}_{lk}} \frac{e^{z^{(2)}_{ij}}}{\sum_m e^{z^{(2)}_{mj}}} = x^{(2)}_{ij} \cdot \frac{\partial}{\partial z^{(2)}_{lk}} log(x^{(2)}_{ij})$

    
    $\frac{\partial}{\partial z^{(2)}_{lk}}log(x^{(2)}_{ij}) = \frac{\partial z^{(2)}_{ij}}{\partial z^{(2)}_{lk}} - \frac{\partial}{\partial z^{(2)}_{lk}}log(\sum_m e^{z^{(2)}_{mj}}) = \frac{\partial z^{(2)}_{ij}}{\partial z^{(2)}_{lk}} - \left[\frac{1}{\sum_m e^{z^{(2)}_{mj}}}\sum_m \frac{\partial e^{z^{(2)}_{mj}}}{\partial z^{(2)}_{lk}}\right] = \frac{\partial z^{(2)}_{ij}}{\partial z^{(2)}_{lk}} - \left[\frac{1}{\sum_m e^{z^{(2)}_{mj}}}\sum_m e^{z^{(2)}_{mj}}\frac{\partial z^{(2)}_{mj}}{\partial z^{(2)}_{lk}}\right]$

    $\frac{\partial X^{(2)}_{ij}}{\partial Z^{(2)}_{lk}} = 
    \begin{cases}
    x^{(2)}_{ij}(1 - x^{(2)}_{ij}) & \text{if } l = i, k = j\\
    -x^{(2)}_{ij}x^{(2)}_{lj} & \text{if } l \neq i, k = j \\
    0 & \text{if } k \neq j
    \end{cases}$

    $\frac{\partial L}{\partial z^{(2)}_{lk}} = -\frac{1}{n}\sum_{j=1}^{n}\sum_{i=1}^{\eta_2}(\frac{y_{ij}}{x_{ij}} \cdot \frac{\partial x^{(2)}_{ij}}{\partial z^{(2)}_{lk}})$, but $\frac{\partial x^{(2)}_{ij}}{\partial z^{(2)}_{lk}}$ only exists when $k = j$ therefore the sum $\sum_ j^n$ is made redundant

    $\implies \frac{\partial L}{\partial Z^{(2)}_{lj}} = -\frac{1}{n}(y_{lj}(1-x^{(2)}_{lj}) + \sum_{i\neq l}-y_{ij}x^{(2)}_{lj}) = -\frac{1}{n}(y_{lj} + \sum_{i}-y_{ij}x^{(2)}_{lj}) = -\frac{1}{n}(y_{lj} + x^{(2)}_{lj}\sum_{i}-y_{ij})$

    $\implies \frac{\partial L}{\partial Z^{(2)}_{lj}} = -\frac{1}{n}(y_{lj} - x^{(2)}_{lj}) = \frac{1}{n}(x^{(2)}_{lj} - y_{lj})$

    $\implies \frac{\partial L}{\partial Z^{(2)}} = \frac{1}{n}(X^{(2)}-Y)$

    $\frac{\partial Z^{(2)}}{\partial A^{(2)}} = \frac{\partial}{\partial A^{(2)}}(A^{(2)}X^{(1)} + B^{(2)}) = X^{(1)T}$

    $\frac{\partial L}{\partial A^{(2)}} = \frac{\partial{L}}{\partial Z^{(2)}} \cdot \frac{\partial Z^{(2)}}{\partial A^{(2)}} = \frac{1}{n}(X^{(2)}-Y)X^{(1)T}$

    For the bias term, we have:

    $\frac{\partial L}{\partial B^{(2)}} = \frac{\partial L}{\partial Z^{(2)}} \cdot \frac{\partial Z^{(2)}}{\partial B^{(2)}}$, where $\frac{\partial Z^{(2)}}{\partial B^{(2)}} = I \in \mathbb{R}^{\eta_2 \times n}$

    However, as $B^{(2)}$ is actually a column vector broadcasted to fit the dimensions, we sum across the columns for the gradient of the bias term.

    $\implies \frac{\partial L}{\partial B^{(2)}} = \frac{\partial L}{\partial Z^{(2)}} = \frac{1}{n}\sum_j(X^{(2)} - Y)$



For the tunable parameters of the first layer, gradient descent requires the following values:

$\frac{\partial L}{\partial A^{(1)}} = \frac{\partial{L}}{\partial X^{(2)}} \cdot \frac{\partial X^{(2)}}{\partial Z^{(2)}} \cdot \frac{\partial Z^{(2)}}{\partial X^{(1)}} \cdot \frac{\partial X^{(1)}}{\partial Z^{(1)}} \cdot  \frac{\partial Z^{(1)}}{\partial A^{(1)}}$;

for the bias term:

$\frac{\partial L}{\partial B^{(1)}} = \frac{\partial{L}}{\partial X^{(2)}} \cdot \frac{\partial X^{(2)}}{\partial Z^{(2)}} \cdot \frac{\partial Z^{(2)}}{\partial X^{(1)}} \cdot \frac{\partial X^{(1)}}{\partial Z^{(1)}} \cdot  \frac{\partial Z^{(1)}}{\partial B^{(1)}}$

We have: 

$\frac{\partial Z^{(2)}}{\partial X^{(1)}} = \frac{\partial}{\partial X^{(1)}}(A^{(2)}X^{(1)}+B^{(2)}) = A^{(2)}$

$\frac{\partial X^{(1)}}{\partial Z^{(1)}}_{ij} = \sigma^{(1)'}(Z^{(1)}) = 
\begin{cases}
1 & \text{if } z_{ij} > 0 \\
0 & \text{otherwise}
\end{cases}$

$\frac{\partial Z^{(1)}}{\partial A^{(1)}} = X^{(0)T}$

$\implies \frac{\partial L}{\partial A^{(1)}} = \frac{1}{n}A^{(2)T}\cdot(X^{(2)}-Y) \odot \sigma^{(1)'}(Z^{(1)}) \cdot X^{(0)T}$

Similarly, for the bias term, we have: 

$\frac{\partial Z^{(1)}}{\partial B^{(1)}} = I \in \mathbb{R}^{\eta_1 \times n}$, where again, the matrix is a column vector broadcasted to fit the dimension.

$\implies \frac{\partial L}{\partial B^{(1)}} = \frac{1}{n}\sum_j(A^{(2)T}\cdot(X^{(2)}-Y) \odot \sigma^{(1)'}(Z^{(1)}))$

Therfore, we have the following for the update steps as part of the gradient descent processes:

$A^{(2)} = A^{(2)} - \alpha \frac{\partial L}{\partial A^{(2)}}$, where $\frac{\partial L}{\partial A^{(2)}} = \frac{1}{n}(X^{(2)}-Y)X^{(1)T}$

$B^{(2)} = B^{(2)} - \alpha\frac{\partial L}{\partial B^{(2)}}$, where $\frac{\partial L}{\partial B^{(2)}} = \frac{1}{n}\sum_j(X^{(2)}-Y)$

$A^{(1)} = A^{(1)} - \alpha \frac{\partial L}{\partial A^{(1)}}$, where $\frac{\partial L}{\partial A^{(1)}} = \frac{1}{n}A^{(2)T}\cdot(X^{(2)}-Y) \odot \sigma^{(1)'}(Z^{(1)}) \cdot X^{(0)T}$

$B^{(1)} = B^{(1)} - \alpha\frac{\partial L}{\partial B^{(1)}}$, where $\frac{\partial L}{\partial B^{(1)}} = \frac{1}{n}\sum_j(A^{(2)T}\cdot(X^{(2)}-Y) \odot \sigma^{(1)'}(Z^{(1)}))$

In [59]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [60]:
data = pd.read_csv('train.csv')

In [61]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
data = np.array(data)
n,p = data.shape                    # we have n = 42000 samples and p = 784 pixels per image
np.random.shuffle(data)             # shuffle to initialise the data
data_dev = data[0:1000].T           # after transposing the first 1000 rows of the dataset,
                                    # the rows become the pixels and the columns are samples
Y_dev = data_dev[0]                 # identify the labels from the data
X_dev = data_dev[1:p]               # identify the pixel data

data_train = data[1000:n].T         # the rest of the data is to train the NN on 
Y_train = data_train[0]             # we do the same
X_train = data_train[1:p]           



In [70]:
def init_params():
    A1 = np.random.randn(10, 784) * np.sqrt(2 / 784)  # He initialization
    B1 = np.zeros((10, 1))
    A2 = np.random.randn(10, 10) * np.sqrt(2 / 10)
    B2 = np.zeros((10, 1))
    return A1, B1, A2, B2

def ReLU(Z):
    return np.maximum(0, Z)                                 # Element-wise comparison to 0

def LeReLU(Z):
    return np.maximum(0.01 * Z, Z)                          # Leaky ReLU to prevent dead neurons

def softmax(x):
    x = x - np.max(x, axis=0, keepdims=True)                # Numerical stability
    e_x = np.exp(x)  
    return e_x / (e_x.sum(axis=0, keepdims=True) + 1e-8)    # np.exp is elementwise exp, and np.sum sums across the axis = 0 means column sum

def forward_prop(A1,B1,A2,B2,X0):                       
    Z1 = A1.dot(X0) + B1                                    # numpy broadcasts the shape of the vector to match the matrix.
    X1 = LeReLU(Z1)
    Z2 = A2.dot(X1) + B2
    X2 = softmax(Z2)
    return Z1,X1,Z2,X2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.max()+1,Y.size))                # create a grid of zeros, rows = classes, columns = samples (1000)
    one_hot_Y[Y, np.arange(Y.size)] = 1                     # np.arange(Y.size) creates an array of entries from 0 to 1000, paired with the label, 
                                                            # it puts a 1 in the correct row and increments the column
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def deriv_LeReLU(Z):
    return np.where(Z > 0, 1, 0.01)                         # Derivative of Leaky ReLU

def back_prop(Z1,X1,Z2,X2,A2,X0,Y):
    n_train = Y.size
    one_hot_Y = one_hot(Y)
    dZ2 = X2 - one_hot_Y
    dA2 = 1 / n_train * dZ2.dot(X1.T)
    dB2 = 1/ n_train * np.sum(dZ2, axis = 1, keepdims= True)                          
    dZ1 = A2.T.dot(dZ2) * deriv_LeReLU(Z1)
    dA1 = 1 / n_train * dZ1.dot(X0.T)
    dB1 = 1/ n_train * np.sum(dZ1,axis=1,keepdims=True)
    return dA1,dB1,dA2,dB2

def update_params(A1,B1,A2,B2,dA1,dB1,dA2,dB2,alpha):
    A1 = A1 - alpha*dA1
    B1 = B1 - alpha*dB1
    A2 = A2 - alpha*dA2
    B2 = B2 - alpha*dB2
    return A1,B1,A2,B2


In [64]:
B1 = np.random.rand(10,1) - 0.5
print(B1.shape)
one_hot_Y = one_hot(Y_dev)

B1 = B1 - 0.05*(np.sum(one_hot_Y,axis=1).reshape(10,1)) 

print(B1)


(10, 1)
[[-4.48531617]
 [-5.45209726]
 [-4.47175194]
 [-6.43592812]
 [-4.74077038]
 [-4.07740524]
 [-4.93070581]
 [-5.15654028]
 [-4.82461889]
 [-5.04494842]]


In [65]:
def get_predictions(X2):
    return np.argmax(X2,0)

def get_accuracy(predictions,Y):
    print(predictions,Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X0, Y, iterations, alpha, decay_rate=0.99):
    A1, B1, A2, B2 = init_params()
    for i in range(iterations):
        Z1, X1, Z2, X2 = forward_prop(A1, B1, A2, B2, X0)
        dA1, dB1, dA2, dB2 = back_prop(Z1, X1, Z2, X2, A2, X0, Y)
        A1, B1, A2, B2 = update_params(A1, B1, A2, B2, dA1, dB1, dA2, dB2, alpha)

        if i % 50 == 0:
            alpha *= decay_rate  # Decay learning rate
            print(f"Iteration {i}, Accuracy: {get_accuracy(get_predictions(X2), Y)}, Alpha: {alpha}")

    return A1, B1, A2, B2

In [71]:
A1,B1,A2,B2 = gradient_descent(X_train,Y_train, 500, 0.01)

[2 2 5 ... 2 8 2] [9 9 5 ... 1 0 9]
Iteration 0, Accuracy: 0.08275609756097561, Alpha: 0.0099
[4 8 8 ... 1 0 8] [9 9 5 ... 1 0 9]
Iteration 50, Accuracy: 0.3333170731707317, Alpha: 0.009801
[4 7 3 ... 1 0 7] [9 9 5 ... 1 0 9]
Iteration 100, Accuracy: 0.5303170731707317, Alpha: 0.00970299
[4 7 5 ... 1 0 7] [9 9 5 ... 1 0 9]
Iteration 150, Accuracy: 0.5310243902439025, Alpha: 0.0096059601
[4 7 3 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 200, Accuracy: 0.6392682926829268, Alpha: 0.009509900499
[4 9 5 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 250, Accuracy: 0.6614390243902439, Alpha: 0.00941480149401
[4 9 5 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 300, Accuracy: 0.7350487804878049, Alpha: 0.0093206534790699
[4 7 5 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 350, Accuracy: 0.7066097560975609, Alpha: 0.0092274469442792
[4 9 5 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 400, Accuracy: 0.7597317073170732, Alpha: 0.009135172474836408
[4 9 3 ... 1 0 9] [9 9 5 ... 1 0 9]
Iteration 450, Accuracy: 0.8349024390243902